this code needs to run in the venv beattrack python 3.11.9, as the madmom files ahve been edited to complied in this environment

In [30]:
pip install torch torchvision torchaudio

  Using cached torch-2.2.2-cp311-none-macosx_10_9_x86_64.whl.metadata (25 kB)
  Using cached torchvision-0.17.2-cp311-cp311-macosx_10_13_x86_64.whl.metadata (6.6 kB)
  Using cached torchaudio-2.2.2-cp311-cp311-macosx_10_13_x86_64.whl.metadata (6.4 kB)
Using cached torch-2.2.2-cp311-none-macosx_10_9_x86_64.whl (150.8 MB)
Using cached torchvision-0.17.2-cp311-cp311-macosx_10_13_x86_64.whl (1.7 MB)
Using cached torchaudio-2.2.2-cp311-cp311-macosx_10_13_x86_64.whl (3.4 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
# 设置模块路径
module_path = os.path.abspath(os.path.join('/Users', 'cassielu', 'Beat-Transformer', 'code'))
print("Module path:", module_path)


Module path: /Users/cassielu/Beat-Transformer/code


In [5]:
# 检查并添加模块路径
import sys
if module_path not in sys.path:
    print(f"Adding path to sys.path: {module_path}")
    sys.path.append(module_path)
else:
    print(f"Path is already in sys.path: {module_path}")

# 检查模块路径是否存在并列出其中的文件
if os.path.exists(module_path):
    print("Files in the module directory:", os.listdir(module_path))
else:
    print(f"Module path does not exist: {module_path}")


Adding path to sys.path: /Users/cassielu/Beat-Transformer/code
Files in the module directory: ['train.sh', 'DilatedTransformerLayer.py', 'eight_fold_test.py', 'DilatedTransformer.py', '__pycache__', 'spectrogram_dataset.py', 'ablation_models', 'utils.py', 'train.py', 'optimizer.py', 'visiualize_attention.py']


In [6]:
from DilatedTransformer import Demixed_DilatedTransformerModel

In [8]:
import numpy as np

# 原始代码
np.float = float    
np.int = int   # module 'numpy' has no attribute 'int'
np.object = object    # module 'numpy' has no attribute 'object'
np.bool = bool    # module 'numpy' has no attribute 'bool'



In [9]:
import IPython.display as ipd
import librosa
import torch

In [10]:
from madmom.features.beats import DBNBeatTrackingProcessor
from madmom.features.downbeats import DBNDownBeatTrackingProcessor


In [11]:
#Initialize Beat Transformer to estimate (down-)beat activation from demixed input
model = Demixed_DilatedTransformerModel(attn_len=5, instr=5, ntoken=2,
                                        dmodel=256, nhead=8, d_hid=1024,
                                        nlayers=9,  norm_first=True)
PARAM_PATH = {
    0: "/Users/cassielu/Beat-Transformer/checkpoint/fold_0_trf_param.pt",
    1: "/Users/cassielu/Beat-Transformer/checkpoint/fold_1_trf_param.pt",
    2: "/Users/cassielu/Beat-Transformer/checkpoint/fold_2_trf_param.pt",
    3: "/Users/cassielu/Beat-Transformer/checkpoint/fold_3_trf_param.pt",
    4: "/Users/cassielu/Beat-Transformer/checkpoint/fold_4_trf_param.pt",
    5: "/Users/cassielu/Beat-Transformer/checkpoint/fold_5_trf_param.pt",
    6: "/Users/cassielu/Beat-Transformer/checkpoint/fold_6_trf_param.pt",
    7: "/Users/cassielu/Beat-Transformer/checkpoint/fold_7_trf_param.pt"
}

#Initialize DBN Beat Tracker to locate beats from beat activation
beat_tracker = DBNBeatTrackingProcessor(min_bpm=55.0, max_bpm=215.0, fps=44100/1024,
                                        transition_lambda=100, observation_lambda=6,
                                        num_tempi=None, threshold=0.2)

#Initialize DBN Downbeat Tracker to locate downbeats from downbeat activation
downbeat_tracker = DBNDownBeatTrackingProcessor(beats_per_bar=[3, 4],
                                                min_bpm=55.0, max_bpm=215.0, fps=44100/1024,
                                                transition_lambda=100, observation_lambda=6,
                                                num_tempi=None, threshold=0.2)

In [12]:
# 打印PyTorch版本
print("PyTorch version:", torch.__version__)

# 检查CUDA是否可用
print("CUDA available:", torch.cuda.is_available())

# 查看CUDA版本
print("CUDA version:", torch.version.cuda)

PyTorch version: 2.2.2
CUDA available: False
CUDA version: None


In [13]:
# 加载保存的数组
x = np.load('/Users/cassielu/Desktop/testdance/test1/sleepter/processed_array.npy')


In [14]:

FOLD = 3
model.load_state_dict(torch.load(PARAM_PATH[FOLD], map_location=torch.device('cpu'))['state_dict'])
# 删除这行与GPU相关的代码
# model.cuda()
model.eval()

with torch.no_grad():
    # 修改这行代码以在CPU上运行
    model_input = torch.from_numpy(x).unsqueeze(0).float()
    activation, _ = model(model_input)

beat_activation = torch.sigmoid(activation[0, :, 0]).detach().numpy()
downbeat_activation = torch.sigmoid(activation[0, :, 1]).detach().numpy()
dbn_beat_pred = beat_tracker(beat_activation)

combined_act = np.concatenate((np.maximum(beat_activation - downbeat_activation,
                                          np.zeros(beat_activation.shape)
                                          )[:, np.newaxis],
                              downbeat_activation[:, np.newaxis]
                               ), axis=-1)   #(T, 2)
dbn_downbeat_pred = downbeat_tracker(combined_act)
dbn_downbeat_pred = dbn_downbeat_pred[dbn_downbeat_pred[:, 1]==1][:, 0]

/opt/anaconda3/envs/beattrack/lib/python3.11/site-packages/madmom/features/beats_hmm.py:631: RuntimeWarning: divide by zero encountered in log
  log_densities[:, 1] = np.log(observations[:, 0])


In [34]:
# 打印DBN预测结果
print("DBN Beat Predictions:", dbn_beat_pred)
print("DBN Downbeat Predictions:", dbn_downbeat_pred)

DBN Beat Predictions: [ 0.39473923  0.7662585   1.13777778  1.50929705  1.85759637  2.2523356
  2.62385488  3.0185941   3.39011338  3.76163265  4.13315193  4.52789116
  4.89941043  5.27092971  5.64244898  6.03718821  6.38548753  6.78022676
  7.12852608  7.52326531  7.89478458  8.26630385  8.61460317  9.0093424
  9.38086168  9.75238095 10.14712018 10.51863946 10.89015873 11.28489796
 11.63319728 12.02793651 12.39945578 12.77097506 13.14249433 13.51401361
 13.88553288 14.25705215 14.62857143 15.0000907  15.37160998 15.76634921
 16.16108844 16.53260771 16.90412698 17.27564626 17.64716553 18.01868481
 18.39020408 18.78494331 19.13324263 19.52798186 19.89950113 20.27102041
 20.61931973 21.03727891 21.38557823 21.78031746 22.15183673 22.52335601
 22.89487528 23.26639456 23.63791383 24.00943311 24.38095238 24.75247166
 25.12399093 25.4955102  25.86702948 26.23854875 26.63328798 27.00480726
 27.39954649 27.77106576 28.14258503 28.53732426 28.90884354 29.28036281
 29.65188209 30.02340136 30.394